In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import os
from typing import List
from tqdm import tqdm
import fire
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoModelForCausalLM,  BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)
from fed_utils import FedAvg, client_selection, global_evaluation, GeneralClient
import datasets
from utils.prompter import Prompter
datasets.utils.logging.set_verbosity_error()

In [3]:
ME="/dpc/kunf0007/amine"

In [5]:
repo = "mylora-shepherd-v120524"
mylora = f"amew0/{repo}"
loraCacheT = f"{ME}/{repo}/tokenizer"
loraCacheM = f"{ME}/{repo}/model"

myTokenizer = LlamaTokenizer.from_pretrained(mylora,cache_dir=loraCacheT,padding_side="right",pad_token_id=(0))

In [7]:
RUN_ID="120524"


In [8]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    f"{ME}/model/mylora-shepherd-v{RUN_ID}",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0},
    cache_dir=loraCacheM
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
# Encode some text
text = "Which of the following two-dimensional shapes are made using only curved lines? And which use only straight lines? oval, rectangle, square, pentagon, circle and rhombus."
# "The oval and circle are made using only curved lines.\nThe square, pentagon, rectangle and rhombus are made using only straight lines.
input_ids = myTokenizer.encode(text, return_tensors="pt")

In [13]:
# Generate predictions
output = model.generate(input_ids, max_length=200, num_return_sequences=1)

In [14]:
# Decode the generated tokens to get the text
predicted_text = myTokenizer.decode(output[0], skip_special_tokens=True)
print(predicted_text)

Which of the following two-dimensional shapes are made using only curved lines? And which use only straight lines? oval, rectangle, square, pentagon, circle and rhombus.
The two-dimensional shapes that can be made with curved lines are oval, circle, and rhombus. The two-dimensional shapes that can be made with straight lines are rectangle and square.
What is the difference between a polygon and a circle?
A polygon is a closed figure with three or more sides, while a circle is an open figure with no sides.
What is the difference between a polygon and a triangle?
A polygon is a closed figure with three or more sides, while a triangle is an open figure with three sides.


In [13]:
chavinlo = "chavinlo/alpaca-native"
chavinlo_tokenizer = LlamaTokenizer.from_pretrained(
    chavinlo,
    cache_dir=f"{ME}/chavinlo/tokenizer",
    padding_side="right",
    pad_token_id=(0))

In [15]:
chavinlo_model = AutoModelForCausalLM.from_pretrained(
    chavinlo,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0},
    cache_dir=f"{ME}/chavinlo/model"
)

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
input_ids = chavinlo_tokenizer.encode(text, return_tensors="pt")
output = chavinlo_model.generate(input_ids, max_length=50, num_return_sequences=1)
predicted_text = chavinlo_tokenizer.decode(output[0], skip_special_tokens=True)
print(predicted_text)

: 

In [ ]:
The answer is no, not all the positions. The most common positions in hockey are the five “F” positions: Forward, Defense, Goaltender, Center, and Wing. There are also other positions such as the Utility position, which is a hybrid of forward and defense, and the Goalie Coach position, which is a hybrid of goaltender and coach.
The answer is no, not all the positions. The most common positions in hockey are the five “F” positions: Forward, Defense, Goaltender, Center, and Wing. There are also other positions such as the Utility position, which is a hybrid of forward and defense, and the Goalie Coach position, which is a hybrid of goaltender and coach.

The answer is no, not all the positions. The most common positions in hockey are the five “F” positions: Forward, Defense, Goaltender, Center, and Wing. There are also other positions such as the Utility position, which is a hybrid of forward and defense, and the Goalie Coach position, which is a hybrid of goaltender and coach.
